### Model Building

The client is launching a new ride sharing program in New York similar to Uber or Lyft. At the end of each trip they want their app to suggest a tip amount to the rider. The company has not acquired any of their own data yet, so they have tasked you with producing a model based off of the taxi data. This model should predict the likely tip amount for a trip based on the other trip attributes. You can assume that the ride sharing company can provide data that has the same attributes as the taxi data for each trip.

In building the model consider the following requirements:

- The model should be built from the taxi dataset. You can supplement the taxi data with external datasets, but this is not a requirement.
- Document your choice of model / algorithm, discussing why you chose it over alternatives.
    - Document how you assess your models performance.
    - Discuss any limitations or caveats of the model which might be an issue in implementing it.
    - Discuss how you might improve your model going forward.
    - Discuss how you might turn this model in to an API the company can use.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, roc_curve

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
df = pd.read_csv('data/data_new-vars.csv')

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'PUdate', 'PUhour', 'DOdate', 'DOhour',
       'trip_duration'],
      dtype='object')

In [5]:
X = df[['RatecodeID', 'trip_distance',
       'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'tolls_amount',
       'total_amount', 'PUhour']]
y = df.tip_amount
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
X_train.head()

,RatecodeID,trip_distance,PULocationID,DOLocationID,fare_amount,extra,tolls_amount,total_amount,PUhour
8371,1,0.74,230,162,5.5,0.0,0.0,6.30,14
5027,1,0.40,68,249,4.0,0.0,0.0,4.80,14
9234,1,0.99,68,164,5.5,0.0,0.0,7.30,14
3944,1,0.90,74,75,5.5,0.0,0.0,7.55,14
6862,1,1.60,162,246,14.5,0.0,0.0,15.30,14


In [12]:
rfc = RandomForestRegressor(random_state=42)
param_grid = { 
    "n_estimators" : [10,20,30],
    'max_features': ['sqrt', 'log2'],
    'criterion' :['mse', 'mae']
}

In [13]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': ['sqrt', 'log2'], 'criterion': ['mse', 'mae']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
CV_rfc.best_params_

{'criterion': 'mae', 'max_features': 'sqrt', 'n_estimators': 30}

In [15]:
rfc_params = RandomForestRegressor(bootstrap=True,
                                    criterion='mae',
                                    max_features='sqrt',
                                    min_samples_split=2,
                                    n_estimators=30)

In [17]:
rfc_params.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
pred = rfc_params.predict(X_test)

In [19]:
print("RMSE for Random Forest on CV data: ",mean_squared_error(y_test, pred))

RMSE for Random Forest on CV data:  1.4028680665151514


I chose a Random Forest model because it performs significally well as a regresor, there's no need for feature normalization, it reduces overfitting, and trains relatively fast. Combined with the grid search for parameters selection, they provide a good regression model without knowing the data thoroughly.

Having said that, xgboost or neural networks usually perform better but they are slower to train and the model engineering needs more work. Also, Random Forest are hard to interpret and usually perform worst than a xgboost on a bigger dataset.

Next steps to improve this model:
    
    - Do feature engineeering.
    - Move from sklearn to TensorFlow or Coffee to customize the model and increase accuracy.
    - Change the model in case the performance is worse as said before.
    
To make a API we would have to use a stream to calculate predictions based on the model used for input. Basically, each time data is upload the model would predict and return those predictions. The model should be trained once a day/week/etc to maintain accuracy depending on how often new data is entered in the system.
This workflow has a component of data cleaning and wrangling for the model to be able to interpret it.